<a href="https://colab.research.google.com/github/Bigdata92/Python_For_DA/blob/master/ch08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Wrangling: Join, Combine, 

In [0]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## 8.1 계층적 색인(Hierarchical Indexing) : 축에 대해 다중 색인 단계 지정

In [2]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [0]:
data.index # 색인 계층 보여줌

In [0]:
data['b'] # 부분 색인 가능
data['b':'c']
data.loc[['b', 'd']]

In [3]:
data.loc[:, 2] # 하위 계층 객체 선택 가능

a    0.478943
c    0.092908
d    0.281746
dtype: float64

In [4]:
data.unstack() # unstack()으로 새롭게 배열 = pivot table 생성 역할 (https://datascienceschool.net/view-notebook/76dcd63bba2c4959af15bec41b197e7c/)

,1,2,3
a,-0.204708,0.478943,-0.519439
b,-0.555730,NaN,1.965781
c,1.393406,0.092908,NaN
d,NaN,0.281746,0.769023


In [5]:
data.unstack().stack() #unstack() 반대 stack()

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [6]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)), # 두 축 모두 계층적 색인 가질수 있음
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [7]:
frame.index.names = ['key1', 'key2'] # key1,key2,state,color 행, 열과 혼동 주의!
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [8]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                       names=['state', 'color']) #multiIndex 재사용 가능

### 8.1.1 계층의 순서 바꾸고 정렬(Reordering and Sorting Levels)

In [9]:
frame.swaplevel('key1', 'key2') #swaplevel - 계층번호, 새로운 객체 반환 

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [10]:
frame.sort_index(level=1) #sort_index - 단일 계층 데이터 정렬 
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### 8.1.2 계층별 요약 통계(Summary Statistics by Level)

In [11]:
frame.sum(level='key2') # level 옵션 - 한 축에 대해서 합을 구하고 싶을때
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### 8.1.3 DataFrame의 컬럼 사용하기 (Indexing with a DataFrame's columns)

In [0]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

In [0]:
frame2 = frame.set_index(['c', 'd']) # set_index - 컬럼을 색인으로 함
frame2

In [0]:
frame.set_index(['c', 'd'], drop=False) # drop=False(명시적으로 남겨둠) 사용시 DataFrame에서 삭제X

In [0]:
frame2.reset_index() # reset_index는 set_index와 반대 개념(색인이 컬럼으로 이동)

## 8.2 데이터 합치기(Combining and Merging Datasets)

### 8.2.1 DataFrame합치기(Database-Style DataFrame Joins)

In [13]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df1
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [14]:
pd.merge(df1, df2) # 다대일 관계 - df1 여러 row, df2 1개 row

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [15]:
pd.merge(df1, df2, on='key') # merge() - inner join같이 df1, df2 모두 존재하는 키 보여줌, 중복된 컬럼 이름 키 사용이 좋음
# https://datascienceschool.net/view-notebook/7002e92653434bc88c8c026c3449d27b/

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [16]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [17]:
pd.merge(df1, df2, how='outer') # how인자로 left,right,outer로 다른 종류 조인 사용 가능

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [0]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})
df1
df2
pd.merge(df1, df2, on='key', how='left')

In [0]:
pd.merge(df1, df2, how='inner') # 다대다 조인 데카르트 곱 반환

In [0]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'], # 여러 개의 키 병합 - 컬럼 이름 리스트 넘기기
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer')

In [0]:
pd.merge(left, right, on='key1')
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

### 8.2.2 색인 병합하기(Merging on Index)

In [0]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
left1
right1
pd.merge(left1, right1, left_on='key', right_index=True)

In [0]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

In [0]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])
lefth
righth

In [0]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True) # 리스트로 여러 개의 컬럼 지정해서 병합
pd.merge(lefth, righth, left_on=['key1', 'key2'],
         right_index=True, how='outer')

In [0]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])
left2
right2
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

In [0]:
left2.join(right2, how='outer') # join() - 컬럼이 겹치지X, 완전히 같거나 유사한 색인 구조 가진 여러 개의 DataFrame 객체 병합에 사용

In [0]:
left1.join(right1, on='key')

In [0]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]], # 색인 대 색인 병합 - DataFrame 리스트를 join()에 넘기면 된다 
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])
another
left2.join([right2, another])
left2.join([right2, another], how='outer')

### 8.2.3 축 따라 이어붙이기(Concatenating Along an Axis)

In [18]:
arr = np.arange(12).reshape((3, 4))
arr
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [0]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [20]:
pd.concat([s1, s2, s3]) #concat() - 값, 색인 연결, axis=0 기본값

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [23]:
pd.concat([s1, s2, s3], axis=1)# axis=1 컬럼단위

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [0]:
s4 = pd.concat([s1, s3])
s4
pd.concat([s1, s4], axis=1)
pd.concat([s1, s4], axis=1, join='inner')

In [0]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']]) #join_axes - 병합 축 직접 지정

In [0]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three']) #계층적 색인 - keys 인자 사용
result
result.unstack()

In [0]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

In [0]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])
df1
df2
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

In [0]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

In [0]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower'])

In [0]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df1
df2

In [0]:
pd.concat([df1, df2], ignore_index=True) #DataFrame의 로우 색인이 필요한 data포함X -> ignore_index=True 옵션 사용

### 8.2.4 겹치는 데이터 합치기(Combining Data with Overlap) - 병합, 이어붙이기 불가능시 사용

In [0]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
a
b
np.where(pd.isnull(a), b, a)

In [0]:
b[:-2].combine_first(a[2:]) #Series객체의 combine_first - 위와 동일 연산 제공 및 데이터 정렬

In [0]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan], #DataFrame에서 combine_first
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})
df1
df2
df1.combine_first(df2)

## 8.3 재형성과 피법(Reshaping and Pivoting) - 표 형식의 data를 재배치하는 다양한 기본 연산

### 8.3.1 계층적 색인으로 재형성하기(Reshaping with Hierarchical Indexing)

In [0]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data

In [0]:
result = data.stack() #stack() - 컬럼 을 로우로

In [0]:
result.unstack() #unstack() - 로우 을 컬럼으로

In [0]:
result.unstack(0)
result.unstack('state')

In [0]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd']) # 해당 레벨에 있는 모든 값이 하위그룹에 속하지X unstack()시 누락data 생길수 있음
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2
data2.unstack()

In [0]:
data2.unstack() #stack() - 누락 data 자동 걸러내서 연산 쉽게 원상 복구 가능
data2.unstack().stack()
data2.unstack().stack(dropna=False)

In [0]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
df
df.unstack('state')

In [0]:
df.unstack('state').stack('side')

### 8.3.2 긴형식에서 넓은 형식으로 피법하기(Pivoting “Long” to “Wide” Format)

In [28]:
data = pd.read_csv('examples/macrodata.csv') # 저장 방법 - 시간순
data.head()
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                         name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D', 'end')
ldata = data.stack().reset_index().rename(columns={0: 'value'})

FileNotFoundError: ignored

In [0]:
ldata[:10] #긴 형식 - 시계열, 둘 이상의 키를 가지는 경우 (각 로우는 단일 관측치)

In [0]:
pivoted = ldata.pivot('date', 'item', 'value') #data, item은 DBMS 관점에서 '기본키'
pivoted

In [0]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

In [0]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]
pivoted['value'][:5]

In [0]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

### 8.3.3 넓은형식에서 긴형식으로 피법하기(Pivoting “Wide” to “Long” Format)

In [0]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

In [0]:
melted = pd.melt(df, ['key']) #pivot과 반대개념 : pandas.melt - 여러 컬럼 병합
melted

In [0]:
reshaped = melted.pivot('key', 'variable', 'value') #pivot 원래 모양
reshaped

In [0]:
reshaped.reset_index()

In [0]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

In [0]:
pd.melt(df, value_vars=['A', 'B', 'C'])
pd.melt(df, value_vars=['key', 'A', 'B'])

## Conclusion